In [10]:
import pandas as pd

In [20]:
pd.set_option('display.float_format', lambda x: '%.9f' % x)



In [21]:
df = pd.read_csv('btc_audit_data.csv')  

In [22]:
df

,height,bestblock,txouts,bogosize,disk_size,total_amount,block_info_unspendable_amount,block_info_total_prevout_spent_amount,block_info_total_new_outputs_ex_coinbase_amount,block_info_coinbase_amount
0,0,000000000019d6689c085ae165831e934ff763ae46a2a6...,0,0,3651350555,0.000000000,50.000000000,0.000000000,0.000000000,50.000000000
1,1,00000000839a8e6886ab5951d76f411475428afc90947e...,1,117,3612710834,50.000000000,0.000000000,0.000000000,0.000000000,50.000000000
2,2,000000006a625f06636b8bb6ac7b960a8d03705d1ace08...,2,234,3612710834,100.000000000,0.000000000,0.000000000,0.000000000,50.000000000
3,3,0000000082b5015589a3fdf2d4baff403e6f0be035a5d9...,3,351,3612710834,150.000000000,0.000000000,0.000000000,0.000000000,50.000000000
4,4,000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c...,4,468,3612710834,200.000000000,0.000000000,0.000000000,0.000000000,50.000000000
...,...,...,...,...,...,...,...,...,...,...
291637,291637,0000000000000000ffdbf77bbd46899c24d32ded0624b9...,9783816,738725053,3469354615,12540789.793902880,0.000000000,1899.565652300,1899.565652300,25.000000000
291638,291638,0000000000000000eb5d7fc13b1837473dff1a3bf47211...,9783669,738714120,3469354615,12540814.793902880,0.000000000,83.582981250,83.582981250,25.000000000
291639,291639,00000000000000004ab74770ea20f5073d2b99fffcff52...,9783535,738704470,3469354615,12540839.793902880,0.000000000,8195.108876210,8195.108876210,25.000000000
291640,291640,0000000000000000db956eb8b78afc6e65127bd587a1b3...,9782941,738660102,3469354615,12540864.793902880,0.000000000,4814.035227070,4814.035227070,25.000000000


In [23]:
# Are there any missing blocks?
seq = pd.RangeIndex(df.height.min(), df.height.max())
len(seq[~seq.isin(df.height)].values) != 0

False

In [24]:
df['total_unspendable'] = df.block_info_unspendable_amount.cumsum()

In [25]:
df['total_issuance'] = df.block_info_coinbase_amount.cumsum()

In [26]:
df['expected_spendable'] = df.total_issuance - df.total_unspendable 

In [27]:
df['reconciliation'] = df.total_issuance - df.total_unspendable - df.total_amount

In [28]:
df[df.block_info_unspendable_amount != 0]

,height,bestblock,txouts,bogosize,disk_size,total_amount,block_info_unspendable_amount,block_info_total_prevout_spent_amount,block_info_total_new_outputs_ex_coinbase_amount,block_info_coinbase_amount,total_unspendable,total_issuance,expected_spendable,reconciliation
0,0,000000000019d6689c085ae165831e934ff763ae46a2a6...,0,0,3651350555,0.000000000,50.000000000,0.000000000,0.000000000,50.000000000,50.000000000,50.000000000,0.000000000,0.000000000
91722,91722,00000000000271a2dc26e7667f8419f2e15416dc6955e5...,63394,6916533,3469354615,4586050.000000000,100.000000000,0.000000000,-50.000000000,50.000000000,150.000000000,4586150.000000000,4586000.000000000,-50.000000000
91812,91812,00000000000af0aed4792b1acee3d966af36cf5def1493...,63357,6911280,3469354615,4590500.000000000,100.000000000,20.000000000,-30.000000000,50.000000000,250.000000000,4590650.000000000,4590400.000000000,-100.000000000
124724,124724,0000000000004c78956f8643262f3622acf22486b12042...,182784,15875025,3469354615,6236099.989999990,0.010000010,165.420000000,165.409999990,50.000000000,250.010000010,6236250.000000000,6235999.989999990,-100.000000000
162705,162705,000000000000007dac86ae0400f08bbcac5e9e2b44bfce...,1265969,96709036,3469354615,8135149.972999990,0.017000000,2849.008959080,2848.991959080,50.000000000,250.027000010,8135300.000000000,8135049.972999990,-100.000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266766,266766,00000000000000025fdacfc7f87bdf4324b192323fd99d...,7362232,556793687,3469354615,11919014.794611810,0.001000000,1869.746276170,1869.746276170,25.000000000,260.205388190,11919175.000000000,11918914.794611810,-100.000000000
268680,268680,00000000000000053b001bec1902df4ed91b966da201a8...,7386124,558581551,3469354615,11966864.794611789,0.000000020,3101.497193140,3101.497193140,25.000000000,260.205388210,11967025.000000000,11966764.794611789,-100.000000000
273648,273648,0000000000000001ea02d3ec1673793b8d8ee478ab845c...,7678673,580518694,3469354615,12091064.793957490,0.000654300,512.073891920,512.073891920,25.000000000,260.206042510,12091225.000000000,12090964.793957490,-100.000000000
283392,283392,0000000000000000f9359f176c14e1f1e9dcd986786d01...,8752086,661065047,3469354615,12334664.793957479,0.000000010,3090.740740670,3090.740740670,25.000000000,260.206042520,12334825.000000000,12334564.793957479,-100.000000000
